In [1]:
# Run magic commands first

%load_ext autoreload
%autoreload 2
%cd ..

/Users/kevinvu/Root/Projects/housing-nyc


In [2]:
import housing_nyc as nyc

see https://stackoverflow.com/questions/4871369/python-error-attributeerror-module-object-has-no-attribute for basics on imports

# Start up of library

In [3]:
psql = nyc.Postgres('nycdb')

In [7]:
from housing_nyc import Postgres

In [8]:
psql = Postgres('nycdb')

In [9]:
psql.__dict__

{'_section_name': 'nycdb',
 'conn': <connection object at 0x11ba60eb0; dsn: 'user=kevinvu password=xxx dbname=nycdb host=localhost port=5432', closed: 0>,
 '_schema_name': 'public'}

In [6]:
help(psql)

Help on Postgres in module housing_nyc.postgres object:

class Postgres(builtins.object)
 |  Postgres(section)
 |  
 |  Base class for managing PostgreSQL connections.
 |  
 |  Parameters
 |  ----------
 |  section : str
 |      Section name in config.ini
 |  
 |  Methods defined here:
 |  
 |  __init__(self, section)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  close(self)
 |  
 |  connect(self)
 |  
 |  connected(self) -> bool
 |  
 |  get(self, table_name, query='')
 |      Imports table into pandas.DataFrame.
 |      
 |      Has an option to form a query statement, else uses standard "select * from table_name"
 |      
 |      Parameters
 |      ----------
 |      table_name : str
 |          Name of table in database
 |      query : str, optional
 |          Custom query statement
 |      
 |      Returns
 |      -------
 |      df : pandas.DataFrame
 |          Imported table
 |  
 |  get_connection_by_config(self)
 |      This method will conn

In [10]:
from housing_nyc.postgres import Nycdb

In [11]:
nycdb = Nycdb()

In [12]:
nycdb.__dict__

{'_section_name': 'nycdb',
 'conn': <connection object at 0x11c012040; dsn: 'user=kevinvu password=xxx dbname=nycdb host=localhost port=5432', closed: 0>,
 '_schema_name': 'public',
 '_new': 'test'}

In [3]:
from housing_nyc import Postgres

In [4]:
psql = Postgres('nycdb')
psql.get('accounts')

InterfaceError: connection already closed

# BblBldg class

In [3]:
test1 = nyc.postgres.BblBldg(mappluto='mappluto_unclipped')

In [6]:
test1

In [38]:
test1.qgis.conn

<connection object at 0x1513962e0; dsn: 'user=kevinvu password=xxx dbname=nycdb host=localhost port=5432', closed: 0>

In [39]:
%%timeit
test1._get_tables()

21.6 s ± 1.54 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
test1.mappluto.head()

,boro,block,lot,bbl,address,zipcode
0,1,00001,0010,1000010010,1 GOVERNORS ISLAND,10004
1,1,00002,0002,1000020002,10 SOUTH STREET,10004
2,1,00003,0010,1000030010,1 SOUTH STREET,10004
3,1,00005,7501,1000057501,125 BROAD STREET,10004
4,1,00006,0001,1000060001,24 SOUTH STREET,10004


In [42]:
test1.avroll.head()

,boro,block,lot,bbl,housenum_lo,housenum_hi,street_name,aptno,zip_code,address
0,1,00001,0010,1000010010,1,1,GOVERNORS ISLAND,,10004,1 GOVERNORS ISLAND
1,1,00001,0101,1000010101,1,1,LIBERTY ISLAND,,10004,1 LIBERTY ISLAND
2,1,00001,0201,1000010201,1,1,ELLIS ISLAND,,10004,1 ELLIS ISLAND
3,1,00001,0301,1000010301,,,JOE DIMAGGIO HIGHWAY,,0,JOE DIMAGGIO HIGHWAY
4,1,00001,0401,1000010401,,,JOE DIMAGGIO HIGHWAY,,0,JOE DIMAGGIO HIGHWAY


## subset avroll

In [118]:
test1.avroll['aptno'].loc[1]

''

In [120]:
len(test1.avroll.loc[test1.avroll['aptno'] != ''])

288183

In [125]:
df3 = test1.avroll.loc[test1.avroll['aptno'] != '']

## create bblbldg table

### join method 1: naked merge

In [45]:
%%timeit
test1._join_tables()

1min 9s ± 8.93 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
test1.bblbldg = test1._join_tables()

In [70]:
display(len(test1.bblbldg))
test1.bblbldg.head()

1028067

,boro,block,lot_x,bbl_x,housenum_lo,housenum_hi,street_name,aptno,zip_code,address_x,lot_y,bbl_y,address_y,zipcode
0,1,00001,0010,1000010010,1,1,GOVERNORS ISLAND,,10004,1 GOVERNORS ISLAND,0010,1000010010,1 GOVERNORS ISLAND,10004
5,1,00001,0101,1000010101,1,1,LIBERTY ISLAND,,10004,1 LIBERTY ISLAND,0101,1000010101,1 LIBERTY ISLAND,10004
7,1,00001,0201,1000010201,1,1,ELLIS ISLAND,,10004,1 ELLIS ISLAND,0201,1000010201,1 ELLIS ISLAND,10004
14,1,00002,0001,1000020001,4,4,SOUTH STREET,,10004,4 SOUTH STREET,0001,1000020001,4 SOUTH STREET,10004
16,1,00002,0002,1000020002,10,10,SOUTH STREET,,10004,10 SOUTH STREET,0002,1000020002,10 SOUTH STREET,10004


### join method 2: with set_index
attempt to speed up join_tables func

replaced
```
dfmerge = self.avroll.merge(self.mappluto, left_on=['boro', 'block', 'zip_code'], right_on=['boro', 'block', 'zipcode'])
```

with

```
df1 = self.mappluto.set_index(['boro', 'block', 'zipcode'])
df2 = self.avroll.set_index(['boro', 'block', 'zip_code'])
df2 = df2.rename_axis(df1.index.names)
dfmerge = df2.join(df1, how='inner', lsuffix='_x', rsuffix='_y')
```

this took a very long time, didn't even finish %%timeit, longer than the former code. the set_index timed at 2 s on avg.

In [108]:
%%timeit
df1 = test1.mappluto.set_index(['boro', 'block', 'zipcode'])
df2 = test1.avroll.set_index(['boro', 'block', 'zip_code'])
df2 = df2.rename_axis(df1.index.names)

2.36 s ± 153 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [137]:
df1 = test1.mappluto.set_index(['boro', 'block', 'zipcode'])
df2 = test1.avroll.set_index(['boro', 'block', 'zip_code'])
df2 = df2.rename_axis(df1.index.names)

In [129]:
display(df1.columns)
display(df2.columns)

Index(['lot', 'bbl', 'address'], dtype='object')

Index(['lot', 'bbl', 'housenum_lo', 'housenum_hi', 'street_name', 'aptno',
       'address'],
      dtype='object')

In [139]:
%%timeit
df2.join(df1, how='inner', lsuffix='_x', rsuffix='_y')

30.3 s ± 2.02 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


better to just use join method 1

## how to use BblBldg

In [ ]:
from housing_nyc import BblBldg

In [37]:
obj = BblBldg(mappluto='mappluto_unclipped')
obj.build_table()
obj.upload(obj.bblbldg, obj.bblbldg_name)

Uploaded table bblbldg in schema public
